# Импорты

In [2]:
import pandas as pd
import json # возможно потом понадобится
from collections import Counter

# Обработка тренировочного датасета

In [5]:
# зададим директорию откуда будет забирать тренировочный набор даннх
file_path_train = 'train.json'

In [6]:
# загружаем наш тренировочный набор данных в pandas
df_train = pd.read_json(file_path_train)

In [7]:
# смотрим сверху - ужас наши юзеры в колонках мы так не привыкли
df_train.head()

,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,user_10,...,user_127746,user_127747,user_127748,user_127749,user_127750,user_127751,user_127752,user_127753,user_127754,user_127755
target,female,female,male,male,female,female,female,female,female,male,...,male,female,male,female,male,male,male,female,female,female
features,"{'orders': [{'site-id': 1, 'orders': [{'create...","{'visits': [{'site-id': 3, 'first-seen': 16962...","{'orders': [{'site-id': 21, 'orders': [{'creat...","{'orders': [{'site-id': 2, 'orders': [{'create...","{'orders': [{'site-id': 39, 'orders': [{'creat...","{'orders': [{'site-id': 123, 'orders': [{'crea...","{'visits': [{'site-id': 225, 'first-seen': 169...","{'orders': [{'site-id': 196, 'orders': [{'crea...","{'orders': [{'site-id': 307, 'orders': [{'crea...","{'orders': [{'site-id': 49, 'orders': [{'creat...",...,"{'orders': [{'site-id': 173, 'orders': [{'crea...","{'visits': [{'site-id': 3, 'first-seen': 16897...","{'orders': [{'site-id': 348, 'orders': [{'crea...","{'orders': [{'site-id': 39, 'orders': [{'creat...","{'orders': [{'site-id': 41, 'orders': [{'creat...","{'orders': [{'site-id': 307, 'orders': [{'crea...","{'orders': [{'site-id': 147, 'orders': [{'crea...","{'orders': [{'site-id': 20, 'orders': [{'creat...","{'orders': [{'site-id': 407, 'orders': [{'crea...","{'orders': [{'site-id': 726, 'orders': [{'crea..."


Отмечам про себя что в наборе у нас 127755 пользователей

In [8]:
# транспонируем датасет что бы он принял человеческий вид
df_transposed = df_train.T

# зададим имена колонок (важно когда будем работать с тестовым набором то колонки таргет у нас не будет)
df_transposed.reset_index(inplace=True)
df_transposed.rename(columns={'index': 'user_id', 0: 'target', 1: 'features'}, inplace=True)

# Посмотрим что получилось
df_transposed.head()


,user_id,target,features
0,user_1,female,"{'orders': [{'site-id': 1, 'orders': [{'create..."
1,user_2,female,"{'visits': [{'site-id': 3, 'first-seen': 16962..."
2,user_3,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
3,user_4,male,"{'orders': [{'site-id': 2, 'orders': [{'create..."
4,user_5,female,"{'orders': [{'site-id': 39, 'orders': [{'creat..."


# Далее мы должны распарсить поле характеристики, там все самое интересное, вот этим и займемся ниже

In [9]:
# найдем все уникальные ключи внути ключа features, что бы понять сколько в нем смысловых блоков из которым мы будем парсить данные
unique_keys = set()
for features in df_transposed['features']:
    unique_keys.update(features.keys()) # просто вытащим их в форме множества

# создадим из них новые колонки извлекая данные из 'features'
for key in unique_keys:
    df_transposed[key] = df_transposed['features'].apply(lambda x: x.get(key)) # да просто обратимся по ключу

# колонку 'features' удалим за дальнейшей ненадобностью
df_transposed.drop('features', axis=1, inplace=True)

# посмотрим что получилось
df_transposed.head()


,user_id,target,orders,site-meta,exchange-sessions,last-visits-in-categories,visits
0,user_1,female,"[{'site-id': 1, 'orders': [{'created-at': 1634...",[{'site-id': 2}],None,None,None
1,user_2,female,None,"[{'site-id': 4}, {'site-id': 5}, {'site-id': 6...",None,"[{'category': 'other', 'last-visit-at': 169627...","[{'site-id': 3, 'first-seen': 1696277805, 'las..."
2,user_3,male,"[{'site-id': 21, 'orders': [{'created-at': 168...","[{'site-id': 24}, {'site-id': 25}, {'site-id':...",None,"[{'category': 'electronics', 'last-visit-at': ...","[{'site-id': 3, 'first-seen': 1696320204, 'las..."
3,user_4,male,"[{'site-id': 2, 'orders': [{'created-at': 1600...","[{'site-id': 33}, {'site-id': 2}, {'site-id': ...",None,"[{'category': 'hypermarket', 'last-visit-at': ...","[{'site-id': 3, 'first-seen': 1693468067, 'las..."
4,user_5,female,"[{'site-id': 39, 'orders': [{'created-at': 164...","[{'site-id': 42}, {'site-id': 43}, {'site-id':...",None,"[{'category': 'insurance', 'last-visit-at': 16...","[{'site-id': 3, 'first-seen': 1693796766, 'las..."


![Описание наших колонок с данными](03ed856b-a2a1-423c-a79a-635b2a9fd5ba.jpg)


## Ну что же приступаем к первому этапу извлечения данных, сначала разберемся с заказами и вытащим из них сайты

In [10]:
# Функция рассчитывает определенные айди сайтов в колонке заказы посчитает сколько
# раз наш пользователь заказывал что то на этом сайте
def count_site_ids_in_orders_modified(orders):
    site_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            if 'orders' in order_dict and 'site-id' in order_dict:
                site_id = order_dict['site-id']
                for order in order_dict['orders']:
                    site_id_counter[site_id] += 1
    return site_id_counter

In [11]:

# Создадим новую колонку в которой будет количество заказов на каждом сайте для каждого пользователя
df_transposed['orders_count'] = df_transposed['orders'].apply(count_site_ids_in_orders_modified)

# посмотрим что получилось
df_transposed[['user_id', 'orders_count']].head()


,user_id,orders_count
0,user_1,"{1: 1, 2: 2}"
1,user_2,{}
2,user_3,"{21: 1, 22: 1}"
3,user_4,"{2: 1, 29: 2, 30: 4, 21: 1}"
4,user_5,"{39: 1, 40: 1, 20: 1}"


In [12]:
# теперь тоже самое сделаем и для колонки с посещениями сайтов - посмотрим куда человек
#  ходит и как часто
def count_site_ids_in_visits_modified(visits):
    site_id_counter = Counter()
    if visits is not None:
        for visit_group in visits:
            if 'visits' in visit_group and 'site-id' in visit_group:
                site_id = visit_group['site-id']
                for visit in visit_group['visits']:
                    site_id_counter[site_id] += 1
    return site_id_counter

In [13]:

# Применим функцию и создадим новый столбец с посещениями сайтов
df_transposed['visits_count'] = df_transposed['visits'].apply(count_site_ids_in_visits_modified)

# посмотрим что получилось
df_transposed[['user_id', 'visits_count']].head()


,user_id,visits_count
0,user_1,{}
1,user_2,{3: 2}
2,user_3,"{3: 1, 23: 3, 16: 10}"
3,user_4,"{3: 2, 21: 1, 20: 4, 31: 1, 30: 6, 32: 3}"
4,user_5,"{3: 3, 23: 1, 13: 7, 41: 2, 20: 10}"


In [14]:
# теперь для каждого пользвателя мы хотим вытащить последние категории посещений
# они не повторяются поэтому нам не нужен словарь обойдемся списком
def extract_categories(last_visits):
    if last_visits is not None:
        return [visit['category'] for visit in last_visits]
    return []

In [15]:
# применим функцию и создадим новый столбец с требуемым списком
df_transposed['last_visited_categories'] = df_transposed['last-visits-in-categories'].apply(extract_categories)

# И сразу проверим что получилось
df_transposed[['user_id', 'last_visited_categories']].head()

,user_id,last_visited_categories
0,user_1,[]
1,user_2,[other]
2,user_3,"[electronics, other]"
3,user_4,"[hypermarket, other, education]"
4,user_5,"[insurance, fashion, other, bank, pets, electr..."


In [16]:
# Функция для извлечения списка site-id из колонки site-meta
# описание довольно размытое, но мета в dote это те герои которые популярны сейчас,
# в текущих реалиях, возмлжно это сайты которые сейчас популярны у пользователя
# это важно стоит потом подумать о весе этого списка (да снова список)
def extract_site_meta(site_meta):
    if site_meta is not None:
        return [meta['site-id'] for meta in site_meta]
    return []


In [17]:
# Применение функции к столбцу site-meta
df_transposed['site_meta_list'] = df_transposed['site-meta'].apply(extract_site_meta)
# проверим что там у нас получилось
df_transposed[['user_id', 'site_meta_list']].head()

,user_id,site_meta_list
0,user_1,[2]
1,user_2,"[4, 5, 6, 7, 8, 9, 10, 11, 12]"
2,user_3,"[24, 25, 26, 21, 16, 17, 27, 28, 22]"
3,user_4,"[33, 2, 34, 35, 30, 32, 21, 36, 20, 29, 17, 3,..."
4,user_5,"[42, 43, 44, 35, 41, 36, 39, 20, 45, 46, 17, 3..."


Мы незаслуженно обошли внииманием дополнительную информацию, которая есть в колонке заказы!!! Нужно это срочно устранить

In [18]:
# эта функция посччитает количество предметов купленных пользователем и сгруппирует их
# по их номеру
def count_item_ids_in_orders(orders):
    item_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    item_id = item.get('id')
                    count = item.get('count', 1)  # Устанавливаем значение по умолчанию 1
                    if count is not None:  # Дополнительная проверка на None
                        item_id_counter[item_id] += count
    return item_id_counter


In [19]:
# эта функция в том же накопительном режиме спарсит категории товаров, которые
# покупал пользователь
def count_categories_in_orders(orders):
    category_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    categories = item.get('general-category-path', [])
                    for category in categories:
                        category_counter[category] += 1
    return category_counter

In [20]:
# Эта функция займется подсчетом бредов, тоже показтельно
def count_brand_ids_in_orders(orders):
    brand_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    brand_id = item.get('brand-id')
                    count = item.get('count', 1)  # Устанавливаем значение по умолчанию 1
                    if brand_id is not None and count is not None:  # Дополнительная проверка на None
                        brand_id_counter[brand_id] += count
    return brand_id_counter


In [21]:
# Применение функции к заказам - вытащим товры
df_transposed['item_ids_count'] = df_transposed['orders'].apply(count_item_ids_in_orders)
df_transposed[['user_id', 'item_ids_count']].head()

,user_id,item_ids_count
0,user_1,{}
1,user_2,{}
2,user_3,{'item_1': 1}
3,user_4,"{'item_8': 1, 'item_9': 1, 'item_10': 1, 'item..."
4,user_5,"{'item_13': 1, 'item_14': 1, 'item_15': 1, 'it..."


In [22]:
# тут выащим категорию товара (! обязательно потом join с табличкой глазами глянуть)
df_transposed['category_path_count'] = df_transposed['orders'].apply(count_categories_in_orders)
df_transposed[['user_id', 'category_path_count']].head()

,user_id,category_path_count
0,user_1,{}
1,user_2,{}
2,user_3,{}
3,user_4,"{90404: 1, 90403: 1, 90402: 1, 90698: 1, 73303..."
4,user_5,"{91491: 1, 91461: 2, 198119: 2, 91072: 1, 9149..."


In [23]:
# ну и бренды, на самом деле они мне кажется мало информативны, но все таки посмотрим
df_transposed['brand_ids_count'] = df_transposed['orders'].apply(count_brand_ids_in_orders)
df_transposed[['user_id', 'brand_ids_count']].head()

,user_id,brand_ids_count
0,user_1,{}
1,user_2,{}
2,user_3,{}
3,user_4,"{3243: 1, 1210537: 1, 1773: 1}"
4,user_5,"{1: 1, 1161: 1, 4315: 1}"


У нас есть еще одна загадочная колонка с предыдущим опытом игры в витрину подарков, это может быть интересно, потому что если я правильно понял, то человек из интересного ему выбирал самое интересное!

In [24]:
# Функция для извлечения списка выбранных сайтов из колонки exchange-sessions
def extract_selected_sites(exchange_sessions):
    selected_sites = []
    if exchange_sessions is not None:
        for session in exchange_sessions:
            accepted_site_id = session.get('accepted-site-id')
            if accepted_site_id is not None:
                selected_sites.append(accepted_site_id)
    return selected_sites


In [25]:
# Применение функции к DataFrame, вытащим что же наши пользователи выбирали
df_transposed['selected_sites'] = df_transposed['exchange-sessions'].apply(extract_selected_sites)

# Проверка результата для первых нескольких пользователей
print(df_transposed[['user_id', 'selected_sites']].head(50))

    user_id                                     selected_sites
0    user_1                                                 []
1    user_2                                                 []
2    user_3                                                 []
3    user_4                                                 []
4    user_5                                                 []
5    user_6                               [207, 1, 21, 34, 16]
6    user_7                                                 []
7    user_8                                                 []
8    user_9                                                 []
9   user_10                                                 []
10  user_11                                                [2]
11  user_12                                                 []
12  user_13                                                 []
13  user_14                                                 []
14  user_15                                            

In [26]:
# посмотрим на набор колонок, который у нас получился
df_transposed.columns

Index(['user_id', 'target', 'orders', 'site-meta', 'exchange-sessions',
       'last-visits-in-categories', 'visits', 'orders_count', 'visits_count',
       'last_visited_categories', 'site_meta_list', 'item_ids_count',
       'category_path_count', 'brand_ids_count', 'selected_sites'],
      dtype='object')

In [27]:
# удалим колонки исходники за ненадобнотстью:)
df_transposed = df_transposed.drop(['visits', 'orders', 'last-visits-in-categories',
       'exchange-sessions', 'site-meta'], axis=1)

In [28]:

# посмотрим на на результат нашего features engeneering
df_transposed.head(20)


,user_id,target,orders_count,visits_count,last_visited_categories,site_meta_list,item_ids_count,category_path_count,brand_ids_count,selected_sites
0,user_1,female,"{1: 1, 2: 2}",{},[],[2],{},{},{},[]
1,user_2,female,{},{3: 2},[other],"[4, 5, 6, 7, 8, 9, 10, 11, 12]",{},{},{},[]
2,user_3,male,"{21: 1, 22: 1}","{3: 1, 23: 3, 16: 10}","[electronics, other]","[24, 25, 26, 21, 16, 17, 27, 28, 22]",{'item_1': 1},{},{},[]
3,user_4,male,"{2: 1, 29: 2, 30: 4, 21: 1}","{3: 2, 21: 1, 20: 4, 31: 1, 30: 6, 32: 3}","[hypermarket, other, education]","[33, 2, 34, 35, 30, 32, 21, 36, 20, 29, 17, 3,...","{'item_8': 1, 'item_9': 1, 'item_10': 1, 'item...","{90404: 1, 90403: 1, 90402: 1, 90698: 1, 73303...","{3243: 1, 1210537: 1, 1773: 1}",[]
4,user_5,female,"{39: 1, 40: 1, 20: 1}","{3: 3, 23: 1, 13: 7, 41: 2, 20: 10}","[insurance, fashion, other, bank, pets, electr...","[42, 43, 44, 35, 41, 36, 39, 20, 45, 46, 17, 3...","{'item_13': 1, 'item_14': 1, 'item_15': 1, 'it...","{91491: 1, 91461: 2, 198119: 2, 91072: 1, 9149...","{1: 1, 1161: 1, 4315: 1}",[]
5,user_6,female,"{123: 3, 50: 8, 21: 2, 150: 3, 146: 1, 151: 3,...","{21: 2, 205: 1, 123: 10, 206: 1, 16: 1, 63: 1,...","[furniture, shoes, fashion, other, bank, trave...","[146, 50, 150, 21, 151, 210, 123, 211, 16, 3, ...","{'item_75': 1, 'item_76': 1, 'item_77': 1, 'it...","{7812065: 1, 7812006: 1, 7811879: 1, 7877999: ...","{2458: 7, 40722: 1, 52667: 3, 34388: 2, 53410:...","[207, 1, 21, 34, 16]"
6,user_7,female,{},"{225: 1, 247: 1, 35: 1, 155: 1, 147: 1, 20: 1,...","[hypermarket, electronics, insurance, other, e...",[20],{},{},{},[]
7,user_8,female,"{196: 6, 62: 3, 35: 3}","{3: 2, 62: 10}","[other, insurance, entertainment_services]","[62, 34, 35, 196, 93, 46]","{'item_141': 2, 'item_142': 1, 'item_143': 1, ...","{10563370: 2, 91298: 2, 90666: 2}",{},[]
8,user_9,female,{307: 1},"{236: 4, 20: 1, 29: 1, 16: 2, 175: 1, 46: 7, 3...","[fashion, other, food, electronics, cosmetics,...","[307, 34, 12, 183]",{},{},{},[]
9,user_10,male,{49: 1},"{3: 1, 13: 1, 39: 1, 208: 3, 171: 10, 129: 1, ...","[bank, electronics, other, household_appliance...","[54, 31, 65]",{},{},{},[]


In [29]:
# вот этот набор даннх можно и сохранить, он будет базовым для дальнейших изысканий
df_transposed.to_csv('8_feature_train_baseline.csv', index=False)

## Второй этап извлечения интересного из данных будем называть создание умных словарей

In [29]:
# Инициализация коэффициентов значимости для каждого столбца - пока подумаем может мы их по другому использовать
coefficients = {
    'orders_count_coeff': 1,
    'visits_count_coeff': 1,
    'last_visited_categories_coeff': 1,
    'site_meta_list_coeff': 1,
    'item_ids_count_coeff': 1,
    'category_path_count_coeff': 1,
    'brand_ids_count_coeff': 1,
    'selected_sites_coeff': 1
}

In [41]:
# стоит попробовать доверьтся данным грид серча
coefficients_greed = {
    'orders_count_coeff': 1.5,
    'visits_count_coeff': 1.5,
    'last_visited_categories_coeff': 1,
    'site_meta_list_coeff': 1,
    'item_ids_count_coeff': 1.5,
    'category_path_count_coeff': 1,
    'brand_ids_count_coeff': 1.5,
    'selected_sites_coeff': 1
}

In [30]:
# Инициализация словарей для каждого типа данных мы вытащим суть из данных в словари
orders_count_stats = {}
visits_count_stats = {}
last_visited_categories_stats = {}
site_meta_list_stats = {}
item_ids_count_stats = {}
category_path_count_stats = {}
brand_ids_count_stats = {}
selected_sites_stats = {}

In [31]:

# Проход по каждой строке DataFrame и мы заполним наши словари в логике ['ключ']:{male:0, female:0}
# по простому посчитаем с сколько сайт, предмет, категория встречается в данных у мужчин и женщин в нашем тренировочном наборе
for _, row in df_transposed.iterrows():
    gender = row['target']
    
    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id not in orders_count_stats:
            orders_count_stats[site_id] = {'male': 0, 'female': 0}
        orders_count_stats[site_id][gender] += count * coefficients['orders_count_coeff']

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id not in visits_count_stats:
            visits_count_stats[site_id] = {'male': 0, 'female': 0}
        visits_count_stats[site_id][gender] += count * coefficients['visits_count_coeff']

    # Обработка last_visited_categories
    for category in row['last_visited_categories']:
        if category not in last_visited_categories_stats:
            last_visited_categories_stats[category] = {'male': 0, 'female': 0}
        last_visited_categories_stats[category][gender] += 1 * coefficients['last_visited_categories_coeff']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id not in site_meta_list_stats:
            site_meta_list_stats[site_id] = {'male': 0, 'female': 0}
        site_meta_list_stats[site_id][gender] += 1 * coefficients['site_meta_list_coeff']

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id not in item_ids_count_stats:
            item_ids_count_stats[item_id] = {'male': 0, 'female': 0}
        item_ids_count_stats[item_id][gender] += count * coefficients['item_ids_count_coeff']

    # Обработка category_path_count
    for category_id, count in row['category_path_count'].items():
        if category_id not in category_path_count_stats:
            category_path_count_stats[category_id] = {'male': 0, 'female': 0}
        category_path_count_stats[category_id][gender] += count * coefficients['category_path_count_coeff']

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id not in brand_ids_count_stats:
            brand_ids_count_stats[brand_id] = {'male': 0, 'female': 0}
        brand_ids_count_stats[brand_id][gender] += count * coefficients['brand_ids_count_coeff']

    # Обработка selected_sites
    for site_id in row['selected_sites']:
        if site_id not in selected_sites_stats:
            selected_sites_stats[site_id] = {'male': 0, 'female': 0}
        selected_sites_stats[site_id][gender] += 1 * coefficients['selected_sites_coeff']


'"0" => "f", "1" => "a", "2" => "s", "3" => "h", "4" => "i", "5" => "o", "6" => "n"': {'male': 0,
  'female': 3} - вот это непонятно что надо на нее посмотреть будет, что это и зачем

ВСЕ НУЖНО ПРОВЕРИТЬ!!!!

In [32]:
# проверяем словарь заказов
orders_count_stats

{1: {'male': 2489, 'female': 1633},
 2: {'male': 49291, 'female': 37842},
 21: {'male': 19100, 'female': 21393},
 22: {'male': 3276, 'female': 3579},
 29: {'male': 31944, 'female': 27778},
 30: {'male': 14101, 'female': 4614},
 39: {'male': 11730, 'female': 11168},
 40: {'male': 19008, 'female': 12275},
 20: {'male': 37843, 'female': 29847},
 123: {'male': 318, 'female': 1652},
 50: {'male': 6834, 'female': 15153},
 150: {'male': 1175, 'female': 2260},
 146: {'male': 591, 'female': 6346},
 151: {'male': 9625, 'female': 2736},
 196: {'male': 3186, 'female': 7246},
 62: {'male': 11361, 'female': 18979},
 35: {'male': 5982, 'female': 4279},
 307: {'male': 1928, 'female': 2191},
 49: {'male': 19693, 'female': 23412},
 389: {'male': 3156, 'female': 1993},
 149: {'male': 1919, 'female': 12074},
 160: {'male': 4324, 'female': 3521},
 179: {'male': 13352, 'female': 34204},
 25: {'male': 2087, 'female': 1267},
 74: {'male': 697, 'female': 449},
 325: {'male': 140, 'female': 381},
 16: {'male': 

In [36]:

# Преобразование в список словарей с добавлением id
list_of_dicts_1 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in orders_count_stats.items()]

# Создание DataFrame заказов
df_orders_dic = pd.DataFrame(list_of_dicts_1)
df_orders_dic.head()


,id,male,female
0,1,2489,1633
1,2,49291,37842
2,21,19100,21393
3,22,3276,3579
4,29,31944,27778


In [37]:
#Сохраним его
df_orders_dic.to_csv('df_orders_dic.csv', index=False)

In [38]:
# проверяем словарь сайтов
visits_count_stats

{3: {'male': 181589, 'female': 195597},
 23: {'male': 41251, 'female': 16308},
 16: {'male': 66892, 'female': 50209},
 21: {'male': 15972, 'female': 13390},
 20: {'male': 108451, 'female': 82323},
 31: {'male': 5728, 'female': 11802},
 30: {'male': 11288, 'female': 2535},
 32: {'male': 1295, 'female': 1688},
 13: {'male': 32463, 'female': 39727},
 41: {'male': 40755, 'female': 28500},
 205: {'male': 3169, 'female': 7544},
 123: {'male': 468, 'female': 2288},
 206: {'male': 19788, 'female': 13515},
 63: {'male': 1790, 'female': 2575},
 207: {'male': 4763, 'female': 4040},
 208: {'male': 7366, 'female': 15912},
 209: {'male': 5204, 'female': 12749},
 225: {'male': 559, 'female': 660},
 247: {'male': 2762, 'female': 3878},
 35: {'male': 5051, 'female': 2874},
 155: {'male': 2952, 'female': 1524},
 147: {'male': 12082, 'female': 5527},
 263: {'male': 738, 'female': 2258},
 10: {'male': 6647, 'female': 12295},
 241: {'male': 2647, 'female': 1484},
 62: {'male': 12165, 'female': 22484},
 236

In [39]:
# Преобразование в список словарей с добавлением id
list_of_dicts_2 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in visits_count_stats.items()]

# Создание DataFrame visits
df_visits_dic = pd.DataFrame(list_of_dicts_2)
df_visits_dic.head()

,id,male,female
0,3,181589,195597
1,23,41251,16308
2,16,66892,50209
3,21,15972,13390
4,20,108451,82323


In [40]:
#Сохраним его
df_visits_dic.to_csv('df_visits_dic.csv', index=False)

In [41]:
last_visited_categories_stats

{'other': {'male': 62416, 'female': 63176},
 'electronics': {'male': 35223, 'female': 30532},
 'hypermarket': {'male': 30217, 'female': 28210},
 'education': {'male': 13687, 'female': 16964},
 'insurance': {'male': 6779, 'female': 5571},
 'fashion': {'male': 16852, 'female': 24432},
 'bank': {'male': 28305, 'female': 22006},
 'pets': {'male': 4366, 'female': 8980},
 'cosmetics': {'male': 13621, 'female': 25971},
 'entertainment_services': {'male': 21604, 'female': 26139},
 'health': {'male': 9191, 'female': 15316},
 'media': {'male': 20004, 'female': 19843},
 'furniture': {'male': 12599, 'female': 17579},
 'shoes': {'male': 4811, 'female': 8511},
 'travel': {'male': 9240, 'female': 12407},
 'household_appliances': {'male': 9481, 'female': 6840},
 'food': {'male': 15524, 'female': 17601},
 'kids': {'male': 6976, 'female': 13786},
 'luxury': {'male': 6532, 'female': 9509},
 'gifts': {'male': 3764, 'female': 6847},
 'dealoftheday': {'male': 2696, 'female': 3614},
 'sport': {'male': 3085, 

In [42]:
# Преобразование в список словарей с добавлением id
list_of_dicts_3 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in last_visited_categories_stats.items()]

# Создание DataFrame по категориям последних посещенных сайтов
df_last_visited_categories_dic = pd.DataFrame(list_of_dicts_3)
df_last_visited_categories_dic.head()

,id,male,female
0,other,62416,63176
1,electronics,35223,30532
2,hypermarket,30217,28210
3,education,13687,16964
4,insurance,6779,5571


In [43]:
#Сохраним его
df_last_visited_categories_dic.to_csv('df_last_visited_categories_dic.csv', index=False)

In [44]:
site_meta_list_stats

{2: {'male': 22853, 'female': 17656},
 4: {'male': 1182, 'female': 12948},
 5: {'male': 420, 'female': 7430},
 6: {'male': 1480, 'female': 5656},
 7: {'male': 8748, 'female': 12895},
 8: {'male': 6953, 'female': 18501},
 9: {'male': 3600, 'female': 6289},
 10: {'male': 4003, 'female': 7072},
 11: {'male': 468, 'female': 1000},
 12: {'male': 2161, 'female': 12102},
 24: {'male': 822, 'female': 540},
 25: {'male': 1662, 'female': 942},
 26: {'male': 1303, 'female': 1289},
 21: {'male': 14044, 'female': 14570},
 16: {'male': 15956, 'female': 14671},
 17: {'male': 33275, 'female': 39693},
 27: {'male': 536, 'female': 499},
 28: {'male': 305, 'female': 330},
 22: {'male': 3087, 'female': 3073},
 33: {'male': 1693, 'female': 1650},
 34: {'male': 4895, 'female': 16553},
 35: {'male': 6736, 'female': 3947},
 30: {'male': 4469, 'female': 1247},
 32: {'male': 239, 'female': 345},
 36: {'male': 5465, 'female': 8631},
 20: {'male': 19806, 'female': 17558},
 29: {'male': 16723, 'female': 16191},
 3

In [46]:
# Преобразование в список словарей с добавлением id
list_of_dicts_4 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in site_meta_list_stats.items()]

# Создание DataFrame по мета-сайтам для пользователя сейчас
df_site_meta_list_dic = pd.DataFrame(list_of_dicts_4)
df_site_meta_list_dic.head()

,id,male,female
0,2,22853,17656
1,4,1182,12948
2,5,420,7430
3,6,1480,5656
4,7,8748,12895


In [47]:
#Сохраним его
df_site_meta_list_dic.to_csv('df_site_meta_list_dic.csv', index=False)

In [47]:
# словарик с товарами
item_ids_count_stats

{'item_1': {'male': 2, 'female': 0},
 'item_8': {'male': 11623, 'female': 9249},
 'item_9': {'male': 1, 'female': 2},
 'item_10': {'male': 2, 'female': 1},
 'item_11': {'male': 4, 'female': 1},
 'item_13': {'male': 1, 'female': 1},
 'item_14': {'male': 0, 'female': 1},
 'item_15': {'male': 0, 'female': 1},
 'item_16': {'male': 0, 'female': 1},
 'item_17': {'male': 1, 'female': 1},
 'item_75': {'male': 0, 'female': 1},
 'item_76': {'male': 0, 'female': 1},
 'item_77': {'male': 0, 'female': 1},
 'item_78': {'male': 0, 'female': 1},
 'item_79': {'male': 0, 'female': 1},
 'item_80': {'male': 0, 'female': 1},
 'item_81': {'male': 0, 'female': 1},
 'item_82': {'male': 1, 'female': 2},
 'item_83': {'male': 0, 'female': 4},
 'item_84': {'male': 0, 'female': 4},
 'item_85': {'male': 0, 'female': 1},
 'item_86': {'male': 15, 'female': 24},
 'item_87': {'male': 2, 'female': 1},
 'item_88': {'male': 0, 'female': 1},
 'item_89': {'male': 0, 'female': 2},
 'item_90': {'male': 3, 'female': 2},
 'item

In [53]:
# Преобразование в список словарей с добавлением id
list_of_dicts_5 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in item_ids_count_stats.items()]

# Создание DataFrame по категориям товаров приобретенных человеком
df_item_ids_count_dic = pd.DataFrame(list_of_dicts_5)
df_item_ids_count_dic.head(10)

,id,male,female
0,item_1,2,0
1,item_8,11623,9249
2,item_9,1,2
3,item_10,2,1
4,item_11,4,1


In [56]:
#Сохраним его
df_item_ids_count_dic.to_csv('df_item_ids_count_dic.csv', index=False)

In [48]:
# ключи вот этого словаря можно поменять на понятные категории, но для идеи нашей модели это не важно, это интересно для аналитики
category_path_count_stats

{90404: {'male': 390, 'female': 284},
 90403: {'male': 845, 'female': 435},
 90402: {'male': 9406, 'female': 4401},
 90698: {'male': 1236, 'female': 1751},
 7330336: {'male': 5018, 'female': 8031},
 90692: {'male': 9244, 'female': 17070},
 90666: {'male': 66697, 'female': 109594},
 8353924: {'male': 568, 'female': 351},
 91497: {'male': 7289, 'female': 5532},
 91461: {'male': 21640, 'female': 15581},
 198119: {'male': 38476, 'female': 26592},
 91491: {'male': 11381, 'female': 7768},
 91072: {'male': 1234, 'female': 1081},
 90508: {'male': 197, 'female': 33},
 13199877: {'male': 612, 'female': 140},
 7812065: {'male': 182, 'female': 1122},
 7812006: {'male': 4250, 'female': 20572},
 7811879: {'male': 12081, 'female': 54363},
 7877999: {'male': 55527, 'female': 196815},
 7811945: {'male': 453, 'female': 2139},
 7811896: {'male': 1471, 'female': 7384},
 7811873: {'male': 8071, 'female': 48124},
 7811903: {'male': 389, 'female': 2774},
 7812156: {'male': 88, 'female': 317},
 7811877: {'mal

In [49]:
# Преобразование в список словарей с добавлением id
list_of_dicts_6 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in category_path_count_stats.items()]

# Создание DataFrame по категориям товаров приобретенных человеком
df_category_path_count_dic = pd.DataFrame(list_of_dicts_6)
df_category_path_count_dic.head()

,id,male,female
0,90404,390,284
1,90403,845,435
2,90402,9406,4401
3,90698,1236,1751
4,7330336,5018,8031


In [50]:
#Сохраним его
df_category_path_count_dic.to_csv('df_category_path_count_dic.csv', index=False)

In [57]:
# словарик с брендами
brand_ids_count_stats


{3243: {'male': 32, 'female': 8},
 1210537: {'male': 64, 'female': 61},
 1773: {'male': 304, 'female': 222},
 1: {'male': 6435, 'female': 4204},
 1161: {'male': 666, 'female': 633},
 4315: {'male': 87, 'female': 53},
 2458: {'male': 266, 'female': 1494},
 40722: {'male': 4, 'female': 28},
 52667: {'male': 3, 'female': 10},
 34388: {'male': 22, 'female': 24},
 53410: {'male': 0, 'female': 2},
 54161: {'male': 28, 'female': 53},
 34399: {'male': 2, 'female': 18},
 36875: {'male': 4, 'female': 8},
 34625: {'male': 10, 'female': 40},
 32338: {'male': 20, 'female': 37},
 35143: {'male': 36, 'female': 42},
 30247: {'male': 6, 'female': 15},
 32312: {'male': 5, 'female': 34},
 38833: {'male': 0, 'female': 3},
 3154: {'male': 744, 'female': 1317},
 3859: {'male': 21, 'female': 366},
 1423: {'male': 170, 'female': 1868},
 3854: {'male': 159, 'female': 1996},
 11725: {'male': 25, 'female': 234},
 2213: {'male': 147, 'female': 37},
 6382: {'male': 20, 'female': 8},
 10472: {'male': 75, 'female': 

In [58]:
# Преобразование в список словарей с добавлением id
list_of_dicts_7 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in brand_ids_count_stats.items()]

# Создание DataFrame по категориям товаров приобретенных человеком
df_brand_ids_count_dic = pd.DataFrame(list_of_dicts_7)
df_brand_ids_count_dic.head()

,id,male,female
0,3243,32,8
1,1210537,64,61
2,1773,304,222
3,1,6435,4204
4,1161,666,633


In [59]:
#Сохраним его
df_brand_ids_count_dic.to_csv('df_brand_ids_count_dic.csv', index=False)

In [60]:
# словарик с выбором в витрине подарков
selected_sites_stats

{207: {'male': 1311, 'female': 1070},
 1: {'male': 658, 'female': 496},
 21: {'male': 9118, 'female': 10259},
 34: {'male': 4723, 'female': 8376},
 16: {'male': 2488, 'female': 2206},
 2: {'male': 1570, 'female': 1620},
 247: {'male': 174, 'female': 527},
 136: {'male': 116, 'female': 1450},
 278: {'male': 391, 'female': 569},
 111: {'male': 223, 'female': 415},
 253: {'male': 32, 'female': 46},
 180: {'male': 944, 'female': 1944},
 126: {'male': 275, 'female': 308},
 37: {'male': 854, 'female': 710},
 50: {'male': 988, 'female': 1999},
 9: {'male': 944, 'female': 1097},
 237: {'male': 2145, 'female': 2290},
 18: {'male': 25, 'female': 61},
 181: {'male': 118, 'female': 506},
 86: {'male': 299, 'female': 1316},
 76: {'male': 100, 'female': 527},
 199: {'male': 942, 'female': 2980},
 42: {'male': 1222, 'female': 3126},
 74: {'male': 509, 'female': 666},
 328: {'male': 129, 'female': 229},
 241: {'male': 334, 'female': 136},
 13: {'male': 4779, 'female': 4267},
 184: {'male': 145, 'femal

In [61]:
# Преобразование в список словарей с добавлением id
list_of_dicts_8 = [{'id': key, 'male': value['male'], 'female': value['female']} for key, value in selected_sites_stats.items()]

# Создание DataFrame по категориям товаров приобретенных человеком
df_selected_sites_dic = pd.DataFrame(list_of_dicts_8)
df_selected_sites_dic.head()

,id,male,female
0,207,1311,1070
1,1,658,496
2,21,9118,10259
3,34,4723,8376
4,16,2488,2206


In [62]:
#Сохраним его
df_selected_sites_dic.to_csv('df_selected_sites_dic.csv', index=False)

## Второй этап создания умных словарей.

### Создание словарей с доминантными признаками

In [33]:
# Создание словарей для определения доминирующего пола для каждого типа данных (парадигма при равных побеждают женщины)
gender_dominant_orders_count = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in orders_count_stats.items()
}
gender_dominant_visits_count = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in visits_count_stats.items()
}
gender_dominant_last_visited_categories = {
    category: 'female' if stats['female'] > stats['male'] else 'male'
    for category, stats in last_visited_categories_stats.items()
}
gender_dominant_site_meta_list = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in site_meta_list_stats.items()
}
gender_dominant_item_ids_count = {
    item_id: 'female' if stats['female'] > stats['male'] else 'male'
    for item_id, stats in item_ids_count_stats.items()
}
gender_dominant_category_path_count = {
    category_id: 'female' if stats['female'] > stats['male'] else 'male'
    for category_id, stats in category_path_count_stats.items()
}
gender_dominant_brand_ids_count = {
    brand_id: 'female' if stats['female'] > stats['male'] else 'male'
    for brand_id, stats in brand_ids_count_stats.items()
}
gender_dominant_selected_sites = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in selected_sites_stats.items()
}


In [34]:
# Вывод результатов для проверки пока записывать их не будем их успех сомнителен
gender_dominant_orders_count


{1: 'male',
 2: 'male',
 21: 'female',
 22: 'female',
 29: 'male',
 30: 'male',
 39: 'male',
 40: 'male',
 20: 'male',
 123: 'female',
 50: 'female',
 150: 'female',
 146: 'female',
 151: 'male',
 196: 'female',
 62: 'female',
 35: 'male',
 307: 'female',
 49: 'female',
 389: 'male',
 149: 'female',
 160: 'male',
 179: 'female',
 25: 'male',
 74: 'male',
 325: 'female',
 16: 'male',
 46: 'female',
 71: 'female',
 208: 'female',
 58: 'female',
 8: 'female',
 34: 'female',
 391: 'female',
 371: 'female',
 117: 'female',
 351: 'female',
 159: 'female',
 96: 'female',
 18: 'female',
 7: 'female',
 613: 'female',
 266: 'female',
 199: 'female',
 81: 'female',
 348: 'male',
 194: 'female',
 209: 'female',
 489: 'male',
 10: 'female',
 13: 'female',
 147: 'male',
 676: 'male',
 441: 'male',
 173: 'female',
 349: 'male',
 346: 'male',
 439: 'female',
 234: 'female',
 203: 'female',
 488: 'male',
 100: 'female',
 54: 'female',
 174: 'female',
 181: 'female',
 153: 'male',
 119: 'female',
 86: '

In [65]:
gender_dominant_visits_count

{3: 'female',
 23: 'male',
 16: 'male',
 21: 'male',
 20: 'male',
 31: 'female',
 30: 'male',
 32: 'female',
 13: 'female',
 41: 'male',
 205: 'female',
 123: 'female',
 206: 'male',
 63: 'female',
 207: 'male',
 208: 'female',
 209: 'female',
 225: 'female',
 247: 'female',
 35: 'male',
 155: 'male',
 147: 'male',
 263: 'female',
 10: 'female',
 241: 'male',
 62: 'female',
 236: 'female',
 29: 'male',
 175: 'female',
 46: 'female',
 246: 'female',
 348: 'male',
 12: 'female',
 65: 'female',
 39: 'male',
 171: 'female',
 129: 'female',
 477: 'female',
 191: 'female',
 320: 'female',
 217: 'male',
 478: 'male',
 158: 'female',
 160: 'male',
 34: 'female',
 74: 'male',
 351: 'female',
 26: 'male',
 112: 'male',
 98: 'female',
 110: 'male',
 118: 'female',
 144: 'female',
 37: 'male',
 199: 'female',
 51: 'female',
 371: 'female',
 146: 'female',
 149: 'female',
 83: 'female',
 103: 'female',
 90: 'female',
 79: 'male',
 136: 'female',
 391: 'female',
 42: 'female',
 495: 'male',
 132: 'f

!!! вот эти словари я еще не пробовал для прогноза, надо будет тоже попробовать - может чем проще тем лучше

### Создание долевых словарей

In [35]:
# Создание словарей для расчета доли мужского и женского участия для каждого типа данных
gender_ratio_orders_count = {}
gender_ratio_visits_count = {}
gender_ratio_last_visited_categories = {}
gender_ratio_site_meta_list = {}
gender_ratio_item_ids_count = {}
gender_ratio_category_path_count = {}
gender_ratio_brand_ids_count = {}
gender_ratio_selected_sites = {}

# Заполнение словарей - вот на них большая надежда
for site_id, stats in orders_count_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_orders_count[site_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for site_id, stats in visits_count_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_visits_count[site_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for category, stats in last_visited_categories_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_last_visited_categories[category] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for site_id, stats in site_meta_list_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_site_meta_list[site_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for item_id, stats in item_ids_count_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_item_ids_count[item_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for category_id, stats in category_path_count_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_category_path_count[category_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for brand_id, stats in brand_ids_count_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_brand_ids_count[brand_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}

for site_id, stats in selected_sites_stats.items():
    total = stats['male'] + stats['female']
    if total > 0:
        gender_ratio_selected_sites[site_id] = {'male': round(stats['male'] / total, 5), 'female': round(stats['female'] / total, 5)}



In [36]:
# Вывод первых пар элементов каждого словаря для проверки
print(list(gender_ratio_orders_count.items())[:3])
print(list(gender_ratio_visits_count.items())[:3])


[(1, {'male': 0.60383, 'female': 0.39617}), (2, {'male': 0.5657, 'female': 0.4343}), (21, {'male': 0.47169, 'female': 0.52831})]
[(3, {'male': 0.48143, 'female': 0.51857}), (23, {'male': 0.71667, 'female': 0.28333}), (16, {'male': 0.57123, 'female': 0.42877})]


То же самое что и выше но через функцию удобнее, пока не выполнять - мешаться будет

In [56]:
# Заполнение словарей для расчета доли мужского и женского участия через фунцию
def calculate_gender_ratio(stats_dict):
    gender_ratio_dict = {}
    for key, stats in stats_dict.items():
        total = stats['male'] + stats['female']
        if total > 0:
            gender_ratio_dict[key] = {
                'male': round(stats['male'] / total, 5),
                'female': round(stats['female'] / total, 5)
            }
    return gender_ratio_dict

gender_ratio_orders_count1 = calculate_gender_ratio(orders_count_stats)
gender_ratio_visits_count1 = calculate_gender_ratio(visits_count_stats)
gender_ratio_last_visited_categories1 = calculate_gender_ratio(last_visited_categories_stats)
gender_ratio_site_meta_list1 = calculate_gender_ratio(site_meta_list_stats)
gender_ratio_item_ids_count1 = calculate_gender_ratio(item_ids_count_stats)
gender_ratio_category_path_count1 = calculate_gender_ratio(category_path_count_stats)
gender_ratio_brand_ids_count1 = calculate_gender_ratio(brand_ids_count_stats)
gender_ratio_selected_sites1 = calculate_gender_ratio(selected_sites_stats)


In [57]:
# Вывод первых пар элементов каждого словаря для проверки - работает и аккуратнее
print(list(gender_ratio_orders_count1.items())[:2])
print(list(gender_ratio_visits_count1.items())[:2])

[(1, {'male': 0.60383, 'female': 0.39617}), (2, {'male': 0.5657, 'female': 0.4343})]
[(3, {'male': 0.48143, 'female': 0.51857}), (23, {'male': 0.71667, 'female': 0.28333})]


В целом на первый взгляд есть в этих правилах вполне понятная логика

А вот тут будет новая идея со значимостью словарей. Нас не устривает сильно нагручивать скор если данный пришли из словаря где 1 мужчина и 0 женщин, нас интересуют реально значимые словари где существено присутствие и мужчин и женщин и самое важное где есть сильный перекос к какую либо сторону

In [37]:
def find_significant_elements(gender_stats_dict, min_interactions=1000, ratio_threshold=1.5):
    significant_elements = set()
    for element, stats in gender_stats_dict.items():
        total_interactions = stats['male'] + stats['female']
        if total_interactions > min_interactions:
            if max(stats['male'], stats['female']) / max(min(stats['male'], stats['female']), 1) >= ratio_threshold:
                significant_elements.add(element)
    return significant_elements

# Применение функции к каждому из словарей gender_stats
significant_orders_count = find_significant_elements(orders_count_stats)
significant_visits_count = find_significant_elements(visits_count_stats)
significant_last_visited_categories = find_significant_elements(last_visited_categories_stats)
significant_site_meta_list = find_significant_elements(site_meta_list_stats)
significant_item_ids_count = find_significant_elements(item_ids_count_stats)
significant_category_path_count = find_significant_elements(category_path_count_stats)
significant_brand_ids_count = find_significant_elements(brand_ids_count_stats)
significant_selected_sites = find_significant_elements(selected_sites_stats)


In [38]:
# Вывод полученных множеств значимых элементов для проверки
print("Significant Orders Count:", significant_orders_count)
print("Significant Visits Count:", significant_visits_count)
print("Significant Last Visited Categories:", significant_last_visited_categories)
print("Significant Site Meta List:", significant_site_meta_list)
print("Significant Item IDs Count:", significant_item_ids_count)
print("Significant Category Path Count:", significant_category_path_count)
print("Significant Brand IDs Count:", significant_brand_ids_count)
print("Significant Selected Sites:", significant_selected_sites)

Significant Orders Count: {1, 5, 6, 7, 8, 10, 11, 12, 18, 23, 25, 30, 34, 40, 42, 44, 45, 46, 556, 50, 51, 565, 54, 55, 53, 57, 58, 62, 65, 71, 74, 76, 77, 590, 79, 80, 84, 86, 87, 89, 91, 93, 606, 95, 96, 97, 98, 99, 100, 104, 105, 106, 110, 111, 114, 118, 123, 130, 132, 645, 646, 135, 136, 650, 141, 653, 146, 147, 148, 149, 150, 151, 153, 155, 158, 159, 162, 167, 172, 173, 174, 175, 178, 179, 692, 181, 180, 183, 184, 182, 192, 193, 194, 195, 196, 197, 198, 199, 206, 719, 208, 209, 214, 728, 234, 244, 247, 263, 266, 311, 317, 319, 324, 337, 338, 340, 341, 344, 345, 350, 361, 389, 407, 439, 441, 485, 488}
Significant Visits Count: {2, 6, 8, 10, 12, 19, 23, 24, 30, 31, 34, 35, 40, 42, 45, 46, 48, 51, 53, 58, 62, 65, 581, 71, 74, 76, 79, 80, 84, 86, 89, 90, 91, 93, 606, 95, 97, 98, 99, 102, 104, 618, 107, 108, 106, 110, 111, 623, 118, 120, 121, 123, 127, 132, 134, 135, 136, 138, 139, 140, 141, 144, 146, 147, 148, 149, 151, 153, 155, 158, 161, 678, 167, 166, 173, 175, 180, 184, 198, 199, 

In [39]:
# Просто посмотрим на количество элементов много таких быть не должно
print("Significant Orders Count:", len(significant_orders_count), 'из общего количество', len(orders_count_stats.items()))
print("Significant Visits Count:", len(significant_visits_count), 'из общего количество', len(visits_count_stats.items()))
print("Significant Last Visited Categories:", len(significant_last_visited_categories), 'из общего количество', len(last_visited_categories_stats.items()))
print("Significant Site Meta List:", len(significant_site_meta_list), 'из общего количество', len(site_meta_list_stats.items()))
print("Significant Item IDs Count:", len(significant_item_ids_count), 'из общего количество', len(item_ids_count_stats.items()))
print("Significant Category Path Count:", len(significant_category_path_count), 'из общего количество', len(category_path_count_stats.items()))
print("Significant Brand IDs Count:", len(significant_brand_ids_count), 'из общего количество', len(brand_ids_count_stats.items()))
print("Significant Selected Sites:", len(significant_selected_sites), 'из общего количество', len(selected_sites_stats.items()))

Significant Orders Count: 127 из общего количество 902
Significant Visits Count: 146 из общего количество 476
Significant Last Visited Categories: 6 из общего количество 24
Significant Site Meta List: 145 из общего количество 836
Significant Item IDs Count: 329 из общего количество 1146346
Significant Category Path Count: 529 из общего количество 3321
Significant Brand IDs Count: 218 из общего количество 32559
Significant Selected Sites: 44 из общего количество 585


## Приступаем к работе над валидационным набором данных

In [40]:
df_val = pd.read_json('val.json')

Тут просто копировал ячейки и менял название датафрейма, описание можно потом поменять ну и вообще обернуть это все в функции

In [42]:
# транспонируем датасет что бы он принял человеческий вид
df_transposed_val = df_val.T

# зададим имена колонок (важно когда будем работать с тестовым набором то колонки таргет у нас не будет)
df_transposed_val.reset_index(inplace=True)
df_transposed_val.rename(columns={'index': 'user_id', 0: 'target', 1: 'features'}, inplace=True)

# Посмотрим что получилось
df_transposed_val.head()

,user_id,target,features
0,user_127756,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."
1,user_127757,male,"{'orders': [{'site-id': 16, 'orders': [{'creat..."
2,user_127758,female,"{'orders': [{'site-id': 149, 'orders': [{'crea..."
3,user_127759,female,"{'orders': [{'site-id': 93, 'orders': [{'creat..."
4,user_127760,female,"{'visits': [{'site-id': 3, 'first-seen': 16836..."


In [43]:
# найдем все унникальные ключи внути ключа features
unique_keys = set()
for features in df_transposed_val['features']:
    unique_keys.update(features.keys())

# создадим из них новые колонки извлекая данные из 'features'
for key in unique_keys:
    df_transposed_val[key] = df_transposed_val['features'].apply(lambda x: x.get(key))

# колонку 'features' удалим за дальнейшей ненадобностью
df_transposed_val.drop('features', axis=1, inplace=True)

# посмотрим что получилось
df_transposed_val.head()

,user_id,target,orders,site-meta,exchange-sessions,last-visits-in-categories,visits
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16...",[{'site-id': 407}],"[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713...","[{'site-id': 3, 'first-seen': 1697125590, 'las..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ...","[{'site-id': 3, 'first-seen': 1698423484, 'las..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...",None,"[{'category': 'sport', 'last-visit-at': 165060...","[{'site-id': 391, 'first-seen': 1685615371, 'l..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16...","[{'site-id': 495, 'first-seen': 1687325784, 'l..."
4,user_127760,female,None,"[{'site-id': 46}, {'site-id': 29}, {'site-id':...",None,"[{'category': 'other', 'last-visit-at': 169765...","[{'site-id': 3, 'first-seen': 1683626530, 'las..."


In [44]:
# Создадим новую колонку в которой будет количество заказов на каждом сайте для каждого пользователя
df_transposed_val['orders_count'] = df_transposed_val['orders'].apply(count_site_ids_in_orders_modified)

# Применим функцию и создадим новый столбец с посещениями сайтов
df_transposed_val['visits_count'] = df_transposed_val['visits'].apply(count_site_ids_in_visits_modified)

# применим функцию и создадим новый столбец с требуемым списком по категориям последних посещений
df_transposed_val['last_visited_categories'] = df_transposed_val['last-visits-in-categories'].apply(extract_categories)

# Применение функции к столбцу site-meta
df_transposed_val['site_meta_list'] = df_transposed_val['site-meta'].apply(extract_site_meta)

# Применение функции к заказам - вытащим товры
df_transposed_val['item_ids_count'] = df_transposed_val['orders'].apply(count_item_ids_in_orders)

# тут выащим категорию товара (! обязательно потом join с табличкой глазами глянуть)
df_transposed_val['category_path_count'] = df_transposed_val['orders'].apply(count_categories_in_orders)

# ну и бренды, на самом деле они мне кажется мало информативны, но все таки посмотрим
df_transposed_val['brand_ids_count'] = df_transposed_val['orders'].apply(count_brand_ids_in_orders)

# Применение функции к DataFrame, вытащим что же наши пользователи выбирали
df_transposed_val['selected_sites'] = df_transposed_val['exchange-sessions'].apply(extract_selected_sites)

In [45]:
# удалим колонки исходники за ненадобнотстью:)
df_transposed_val = df_transposed_val.drop(['visits', 'orders', 'last-visits-in-categories',
       'exchange-sessions', 'site-meta'], axis=1)

In [46]:
df_transposed_val.head()

,user_id,target,orders_count,visits_count,last_visited_categories,site_meta_list,item_ids_count,category_path_count,brand_ids_count,selected_sites
0,user_127756,female,{407: 1},"{3: 2, 407: 10}","[other, education]",[407],{},{},{},"[262, 169]"
1,user_127757,male,"{16: 10, 40: 1, 288: 1, 21: 2}","{3: 1, 308: 1, 16: 1}","[electronics, other, shoes]","[16, 29, 21, 40, 212]","{'item_3780263': 2, 'item_5403882': 2, 'item_1...","{90578: 3, 90574: 3, 198118: 5, 91498: 1, 9149...","{1237: 5, 1: 2, 3519: 1, 996: 1, 1055: 3, 1650...",[42]
2,user_127758,female,"{149: 2, 25: 1, 21: 2, 100: 1, 474: 1}","{391: 1, 42: 7, 485: 1, 820: 1, 97: 5, 39: 1, ...","[sport, furniture, fashion, other, bank, house...","[391, 42, 485, 25, 149, 21, 100, 398, 54, 17, ...","{'item_5409213': 1, 'item_1718311': 2, 'item_1...","{13243353: 1, 91182: 1, 91157: 1, 90509: 4, 76...","{62479: 1, 16627: 4, 595: 2, 9851: 4, 50651: 2...",[]
3,user_127759,female,"{93: 8, 134: 1, 103: 1}","{495: 1, 34: 9, 484: 1, 334: 1, 205: 1, 31: 1,...","[furniture, fashion, other, food, electronics,...","[93, 348, 495, 103, 134, 42, 17, 68]","{'item_508120': 1, 'item_4523963': 1, 'item_93...","{4854062: 3, 91182: 5, 91157: 20, 90509: 20, 8...","{3192: 23, 7066: 1, 3860: 1}","[205, 131]"
4,user_127760,female,{},"{3: 2, 34: 1}","[other, cosmetics]","[46, 29, 34, 12, 93, 42, 76, 17]",{},{},{},[]


In [97]:
# вот этот набор даннх можно и сохранить, он будет базовым валидационным набором для дальнейших изысканий
df_transposed_val.to_csv('8_feature_val_baseline.csv', index=False)

# Переходим к предсказаниям

Вот с этой функцией можно поиграть что то отключить,  а может и что то добавить.

In [193]:
# Создание столбца predict_gender - это самая первая придуманная мною функиция
def predict_gender(row):
    male_score = 0
    female_score = 0

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count
            female_score += gender_ratio_orders_count[site_id]['female'] * count

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count
            female_score += gender_ratio_visits_count[site_id]['female'] * count

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_ratio_last_visited_categories:
    #        male_score += gender_ratio_last_visited_categories[category]['male']
    #        female_score += gender_ratio_last_visited_categories[category]['female']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male']
            female_score += gender_ratio_site_meta_list[site_id]['female']

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count
            female_score += gender_ratio_item_ids_count[item_id]['female'] * count

    # Обработка category_path_count
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] * count

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male']
    #        female_score += gender_ratio_selected_sites[site_id]['female']

    return 'female' if female_score > male_score * 1.39 else 'male'


Методом научного тыка мы установили что в таком варинате максимальный скор мы получаем отключив указанные метрики в подсчете, поздже мы подтвердили догадку запустив простенький вариант грид серча

In [125]:
# Создание столбца predict_gender тут мы выведем скор
def predict_gender_score(row):
    male_score = 0
    female_score = 0
    items_count = 0

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count
            female_score += gender_ratio_orders_count[site_id]['female'] * count
            items_count +=1

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count
            female_score += gender_ratio_visits_count[site_id]['female'] * count
            items_count +=1

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_ratio_last_visited_categories:
    #        male_score += gender_ratio_last_visited_categories[category]['male']
    #        female_score += gender_ratio_last_visited_categories[category]['female']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male']
            female_score += gender_ratio_site_meta_list[site_id]['female']
            items_count +=1

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count
            female_score += gender_ratio_item_ids_count[item_id]['female'] * count
            items_count +=1

    # Обработка category_path_count
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] * count
            items_count +=1

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male']
    #        female_score += gender_ratio_selected_sites[site_id]['female']

    return {'male_score':male_score, 'female_score':female_score, 'items_cout':items_count, 'diff':male_score-female_score}

In [185]:
#второй вариант функции с доминантным полом
def predict_gender_dominant(row):
    male_score = 0
    female_score = 0

    # Обработка orders_count
    for site_id in row['orders_count'].keys():
        if site_id in gender_dominant_orders_count:
            dominant_gender = gender_dominant_orders_count[site_id]
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка visits_count
    for site_id in row['visits_count'].keys():
        if site_id in gender_dominant_visits_count:
            dominant_gender = gender_dominant_visits_count[site_id]
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_dominant_last_visited_categories:
    #        dominant_gender = gender_dominant_last_visited_categories[category]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_dominant_site_meta_list:
            dominant_gender = gender_dominant_site_meta_list[site_id]
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка item_ids_count
    for item_id in row['item_ids_count'].keys():
        if item_id in gender_dominant_item_ids_count:
            dominant_gender = gender_dominant_item_ids_count[item_id]
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка category_path_count
    #for category_id in row['category_path_count'].keys():
    #    if category_id in gender_dominant_category_path_count:
    #        dominant_gender = gender_dominant_category_path_count[category_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка brand_ids_count
    for brand_id in row['brand_ids_count'].keys():
        if brand_id in gender_dominant_brand_ids_count:
            dominant_gender = gender_dominant_brand_ids_count[brand_id]
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_dominant_selected_sites:
    #        dominant_gender = gender_dominant_selected_sites[site_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    return 'female' if female_score > male_score * 1.7 else 'male'

In [50]:
#второй вариант функции с доминантным полом тут мы выведем скор
def predict_gender_dominant_score(row):
    male_score = 0
    female_score = 0
    items_count = 0

    # Обработка orders_count
    for site_id in row['orders_count'].keys():
        if site_id in gender_dominant_orders_count:
            dominant_gender = gender_dominant_orders_count[site_id]
            items_count +=1
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка visits_count
    for site_id in row['visits_count'].keys():
        if site_id in gender_dominant_visits_count:
            dominant_gender = gender_dominant_visits_count[site_id]
            items_count +=1
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_dominant_last_visited_categories:
    #        dominant_gender = gender_dominant_last_visited_categories[category]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_dominant_site_meta_list:
            dominant_gender = gender_dominant_site_meta_list[site_id]
            items_count +=1
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка item_ids_count
    for item_id in row['item_ids_count'].keys():
        if item_id in gender_dominant_item_ids_count:
            dominant_gender = gender_dominant_item_ids_count[item_id]
            items_count +=1
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка category_path_count
    #for category_id in row['category_path_count'].keys():
    #    if category_id in gender_dominant_category_path_count:
    #        dominant_gender = gender_dominant_category_path_count[category_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка brand_ids_count
    for brand_id in row['brand_ids_count'].keys():
        if brand_id in gender_dominant_brand_ids_count:
            dominant_gender = gender_dominant_brand_ids_count[brand_id]
            items_count +=1
            if dominant_gender == 'male':
                male_score += 1
            else:
                female_score += 1

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_dominant_selected_sites:
    #        dominant_gender = gender_dominant_selected_sites[site_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    return {'male_score':male_score, 'female_score':female_score, 'items_cout':items_count, 'diff':male_score-female_score}

Сделаем небольшую заготовочку для проверки 2 функций

In [142]:
df_check = pd.DataFrame()
df_check['id'] = df_transposed_val['user_id']
df_check['real_gender'] = df_transposed_val['target']

In [143]:
df_check.head()

,id,real_gender
0,user_127756,female
1,user_127757,male
2,user_127758,female
3,user_127759,female
4,user_127760,female


In [144]:
df_check_dominant = pd.DataFrame()
df_check_dominant['id'] = df_transposed_val['user_id']
df_check_dominant['real_gender'] = df_transposed_val['target']

In [145]:
df_check_dominant.head()

,id,real_gender
0,user_127756,female
1,user_127757,male
2,user_127758,female
3,user_127759,female
4,user_127760,female


А вот собственно и этап проверки

In [194]:
# Применение функции к DataFrame
df_check['predict_gender_1st'] = df_transposed_val.apply(predict_gender, axis=1)

In [195]:
# Применение функции к DataFrame
df_check['predict_gender_stats'] = df_transposed_val.apply(predict_gender_score, axis=1)

А вот предсказание по доманантным признакам

In [189]:
df_check_dominant['predict_gender_dominant'] = df_transposed_val.apply(predict_gender_dominant, axis=1)

In [190]:
df_check_dominant['predict_gender_dominant_stats'] = df_transposed_val.apply(predict_gender_dominant_score, axis=1)

In [173]:
df_check.head()

,id,real_gender,predict_gender_1st,predict_gender_stats
0,user_127756,female,female,"{'male_score': 4.376469999999999, 'female_scor..."
1,user_127757,male,male,"{'male_score': 24.43843, 'female_score': 17.56..."
2,user_127758,female,female,"{'male_score': 64.17067999999999, 'female_scor..."
3,user_127759,female,female,"{'male_score': 26.098200000000006, 'female_sco..."
4,user_127760,female,female,"{'male_score': 3.31741, 'female_score': 7.6825..."


In [174]:
df_check_dominant.head()

,id,real_gender,predict_gender_dominant,predict_gender_dominant_stats
0,user_127756,female,female,"{'male_score': 0, 'female_score': 4, 'items_co..."
1,user_127757,male,male,"{'male_score': 18, 'female_score': 7, 'items_c..."
2,user_127758,female,female,"{'male_score': 24, 'female_score': 48, 'items_..."
3,user_127759,female,female,"{'male_score': 6, 'female_score': 59, 'items_c..."
4,user_127760,female,female,"{'male_score': 1, 'female_score': 9, 'items_co..."


In [152]:
def calculate_accuracy(df, target, candidate):
    correct_predictions = df[df[target] == df[candidate]].shape[0]
    total_predictions = df.shape[0]
    accuracy = correct_predictions / total_predictions
    return accuracy


In [196]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check, 'real_gender', 'predict_gender_1st')
accuracy

0.7602652384595766

In [197]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_dominant, 'real_gender', 'predict_gender_dominant')
accuracy

0.7588807519947536

Предсказания выше мы получили путем абстрагирования от лишнего, что же это неплохо у нас ниже еще 2 идеи:)

Стоит сохранить получившиеся наборы в excel для дальнейшего анализа

In [202]:
df_check.to_csv('baseline.csv', index=False)

In [201]:
df_check_dominant.to_csv('dominant.csv', index=False)

## Блок грид серча

In [276]:
# вот эта функция с коэффициентами для грид серча
def predict_gender_grid(row, coeffs):
    male_score = 0
    female_score = 0

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count * coeffs['orders_count']
            female_score += gender_ratio_orders_count[site_id]['female'] * count * coeffs['orders_count']

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count * coeffs['visits_count']
            female_score += gender_ratio_visits_count[site_id]['female'] * count * coeffs['visits_count']

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_ratio_last_visited_categories:
    #        male_score += gender_ratio_last_visited_categories[category]['male'] * coeffs['last_visited_categories']
    #        female_score += gender_ratio_last_visited_categories[category]['female'] * coeffs['last_visited_categories']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male'] * coeffs['site_meta_list']
            female_score += gender_ratio_site_meta_list[site_id]['female'] * coeffs['site_meta_list']

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count * coeffs['item_ids_count']
            female_score += gender_ratio_item_ids_count[item_id]['female'] * count * coeffs['item_ids_count']

    # Обработка category_path_count
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count * coeffs['category_path_count']
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count * coeffs['category_path_count']

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count * coeffs['brand_ids_count']
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] * count * coeffs['brand_ids_count']

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male'] * coeffs['selected_sites']
    #        female_score += gender_ratio_selected_sites[site_id]['female'] * coeffs['selected_sites']

    return 'female' if female_score >= male_score else 'male'


In [277]:
def calculate_accuracy(df):
    correct_predictions = df[df['target'] == df['predict_gender']].shape[0]
    total_predictions = df.shape[0]
    accuracy = correct_predictions / total_predictions
    return accuracy

In [274]:
import itertools

# Определение диапазонов коэффициентов
coeff_ranges = {
    'orders_count': [0.5, 1, 1.5],
    'visits_count': [0.5, 1, 1.5],
    'last_visited_categories': [0.5, 1, 1.5],
    'site_meta_list': [0.5, 1, 1.5],
    'item_ids_count': [0.5, 1, 1.5],
    'category_path_count': [0.5, 1, 1.5],
    'brand_ids_count': [0.5, 1, 1.5],
    'selected_sites': [0.5, 1, 1.5]
}

# Создание всех возможных комбинаций коэффициентов
all_combinations = itertools.product(*(coeff_ranges[name] for name in coeff_ranges))

# Перебор всех комбинаций и вычисление точности для каждой
best_accuracy = 0
best_coeffs = None
for coeffs in all_combinations:
    coeffs_dict = dict(zip(coeff_ranges.keys(), coeffs))
    
    # Применение функции predict_gender с текущими коэффициентами
    df_transposed_val['predict_gender'] = df_transposed_val.apply(lambda row: predict_gender(row, coeffs_dict), axis=1)

    # Вычисление точности
    current_accuracy = calculate_accuracy(df_transposed_val)

    # Обновление лучших коэффициентов при необходимости
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_coeffs = coeffs_dict

# Вывод лучших коэффициентов и соответствующей точности
print("Лучшая точность:", best_accuracy)
print("Лучшие коэффициенты:", best_coeffs)


Лучшая точность: 0.7156337668962
Лучшие коэффициенты: {'orders_count': 1.5, 'visits_count': 1.5, 'last_visited_categories': 0.5, 'site_meta_list': 1, 'item_ids_count': 1.5, 'category_path_count': 0.5, 'brand_ids_count': 1.5, 'selected_sites': 0.5}


Вот это хорошее числовое подтверждение значимости параметров по которым мы смотрим, все что больше 1 полезно, все что меньше нас путает и снижает скор

Что же это стоит проверить:)

In [77]:
# вот эта функция с коэффициентами после грида
def predict_gender_grid(row):
    male_score = 0
    female_score = 0

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count * 1.5
            female_score += gender_ratio_orders_count[site_id]['female'] * count * 1.5

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count * 1.5
            female_score += gender_ratio_visits_count[site_id]['female'] * count * 1.5

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in gender_ratio_last_visited_categories:
    #        male_score += gender_ratio_last_visited_categories[category]['male'] * coeffs['last_visited_categories']
    #        female_score += gender_ratio_last_visited_categories[category]['female'] * coeffs['last_visited_categories']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male'] * 1
            female_score += gender_ratio_site_meta_list[site_id]['female'] * 1

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count * 1.5
            female_score += gender_ratio_item_ids_count[item_id]['female'] #* count * 1.5

    # Обработка category_path_count
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count * coeffs['category_path_count']
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count * coeffs['category_path_count']

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count * 1.5
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] #* count * 1.5

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male'] * coeffs['selected_sites']
    #        female_score += gender_ratio_selected_sites[site_id]['female'] * coeffs['selected_sites']

    return 'female' if female_score > male_score else 'male'

In [78]:
df_check_greed = pd.DataFrame()
df_check_greed['id'] = df_transposed_val['user_id']
df_check_greed['real_gender'] = df_transposed_val['target']

In [79]:
# Применение функции к DataFrame
df_check_greed['predict_gender_grid'] = df_transposed_val.apply(predict_gender_grid, axis=1)

In [80]:
def calculate_accuracy(df, target, candidate):
    correct_predictions = df[df[target] == df[candidate]].shape[0]
    total_predictions = df.shape[0]
    accuracy = correct_predictions / total_predictions
    return accuracy

In [81]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_greed, 'real_gender', 'predict_gender_grid')
accuracy

0.7354173498014355

Итак мы поняли, что пока никакого дополнительного скора нам грид серч не дал, ничего у нас еще полно идей

In [278]:
# Определение диапазонов коэффициентов - вот этот блок выполняется уже 2 день на отдельном компе
coeff_ranges = {
    'orders_count': [0.9, 1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2],
    'visits_count': [0.9, 1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2],
    #'last_visited_categories': [0.5, 1, 1.5],
    'site_meta_list': [0.9, 1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2],
    'item_ids_count': [0.9, 1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2],
    #'category_path_count': [0.5, 1, 1.5],
    'brand_ids_count': [0.9, 1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2],
    #'selected_sites': [0.5, 1, 1.5]
}

# Создание всех возможных комбинаций коэффициентов
all_combinations = itertools.product(*(coeff_ranges[name] for name in coeff_ranges))

# Перебор всех комбинаций и вычисление точности для каждой
best_accuracy = 0
best_coeffs = None
for coeffs in all_combinations:
    coeffs_dict = dict(zip(coeff_ranges.keys(), coeffs))
    
    # Применение функции predict_gender с текущими коэффициентами
    df_transposed_val['predict_gender'] = df_transposed_val.apply(lambda row: predict_gender_grid(row, coeffs_dict), axis=1)

    # Вычисление точности
    current_accuracy = calculate_accuracy(df_transposed_val)

    # Обновление лучших коэффициентов при необходимости
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_coeffs = coeffs_dict

# Вывод лучших коэффициентов и соответствующей точности
print("Лучшая точность:", best_accuracy)
print("Лучшие коэффициенты:", best_coeffs)

БЛОК ЗНАЧИМОСТИ ЭЛЕМЕНТА

Тут мы будем работать со словарями значимости элемента которые мы готовили ранее, мы будем поощерять скор если он есть в значимых элементах и наказывать если нет

In [ ]:
# у нас есть вот такие рейтинковые словари
gender_dominant_orders_count 
gender_dominant_visits_count
gender_dominant_last_visited_categories 
gender_dominant_site_meta_list 
gender_dominant_item_ids_count 
gender_dominant_category_path_count
gender_dominant_brand_ids_count 
gender_dominant_selected_sites
# И вот такие вот множества значимых элементов
significant_orders_count
significant_visits_count 
significant_last_visited_categories 
significant_site_meta_list 
significant_item_ids_count 
significant_category_path_count 
significant_brand_ids_count 
significant_selected_sites


In [372]:
# Создание столбца predict_gender_wise учитывет значимость элемента
def predict_gender_wise(row):
    male_score = 0
    female_score = 0
    significant_multiplier = 2.15  # Усиление значимого скора
    penalty_multiplier = 0.4    # Штраф незначимого скора
    coef = 1

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in significant_orders_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count * coef
            female_score += gender_ratio_orders_count[site_id]['female'] * count * coef

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in significant_visits_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count * coef
            female_score += gender_ratio_visits_count[site_id]['female'] * count * coef

    # Обработка last_visited_categories - лучше убрать!
    for category in row['last_visited_categories']:
        if category in significant_last_visited_categories:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if category in gender_ratio_last_visited_categories:
            male_score += gender_ratio_last_visited_categories[category]['male'] * coef
            female_score += gender_ratio_last_visited_categories[category]['female'] * coef

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in significant_site_meta_list:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male'] * coef
            female_score += gender_ratio_site_meta_list[site_id]['female'] * coef

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if site_id in significant_item_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count * coef
            female_score += gender_ratio_item_ids_count[item_id]['female'] * count * coef

    # Обработка category_path_count - это тоже лучше убрать
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in significant_category_path_count:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count * coef
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count * coef

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if site_id in significant_brand_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count * coef
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] * count * coef

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in significant_selected_sites:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male'] * coef
    #        female_score += gender_ratio_selected_sites[site_id]['female'] * coef

    return 'female' if female_score > male_score * 1.49 else 'male'

In [260]:
df_check_wise = pd.DataFrame()
df_check_wise['id'] = df_transposed_val['user_id']
df_check_wise['real_gender'] = df_transposed_val['target']

In [314]:
# Применение функции к DataFrame
df_check_wise['predict_gender_wise'] = df_transposed_val.apply(predict_gender_wise, axis=1)

In [315]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_wise, 'real_gender', 'predict_gender_wise')
accuracy

0.7884650417167632

Ну что же мы прибавили 3 с лишним пункта и сравнялись с моделями обученными в порядке машинного обучения!!!

А почему бы не попробовать этот фокус с нашей функцией с доминантным полом? => а попробовать надо!

In [ ]:
significant_orders_count
significant_visits_count 
significant_last_visited_categories 
significant_site_meta_list 
significant_item_ids_count 
significant_category_path_count 
significant_brand_ids_count 
significant_selected_sites

In [369]:
#второй вариант функции с доминантным полом
def predict_gender_dominant_wise(row):
    male_score = 0
    female_score = 0
    significant_multiplier = 2  # Усиление значимого скора
    penalty_multiplier = 0.5    # Штраф незначимого скора
    coef = 1

    # Обработка orders_count
    for site_id in row['orders_count'].keys():
        if site_id in significant_orders_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_orders_count:
            dominant_gender = gender_dominant_orders_count[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка visits_count
    for site_id in row['visits_count'].keys():
        if site_id in significant_visits_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_visits_count:
            dominant_gender = gender_dominant_visits_count[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка last_visited_categories
    for category in row['last_visited_categories']:
        if category in significant_last_visited_categories:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if category in gender_dominant_last_visited_categories:
            dominant_gender = gender_dominant_last_visited_categories[category]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in significant_site_meta_list:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_site_meta_list:
            dominant_gender = gender_dominant_site_meta_list[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка item_ids_count
    for item_id in row['item_ids_count'].keys():
        if item_id in significant_item_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if item_id in gender_dominant_item_ids_count:
            dominant_gender = gender_dominant_item_ids_count[item_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка category_path_count
    #for category_id in row['category_path_count'].keys():
    #    if category_id in gender_dominant_category_path_count:
    #        dominant_gender = gender_dominant_category_path_count[category_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка brand_ids_count
    for brand_id in row['brand_ids_count'].keys():
        if brand_id in significant_brand_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if brand_id in gender_dominant_brand_ids_count:
            dominant_gender = gender_dominant_brand_ids_count[brand_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_dominant_selected_sites:
    #        dominant_gender = gender_dominant_selected_sites[site_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    return 'female' if female_score > male_score * 2 else 'male'

In [321]:
df_check_wise_dominant = pd.DataFrame()
df_check_wise_dominant['id'] = df_transposed_val['user_id']
df_check_wise_dominant['real_gender'] = df_transposed_val['target']

In [370]:
# Применение функции к DataFrame
df_check_wise_dominant['predict_gender_wise_dominant'] = df_transposed_val.apply(predict_gender_dominant_wise, axis=1)

In [371]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_wise_dominant, 'real_gender', 'predict_gender_wise_dominant')
accuracy

0.7767333406201042

Неплохо, но все же хуже чем предыдущий вариант с долями

Нужно будет внимательно посмотреть на пользователей по которым мы ошиблись

## Что планируем:
1) докрутить функцию со значением, посмотреть как на результат будут влиять доли значимости, которые мы определяем в словарях
2) попробовать реализовать в ней штраф и поощерение не только общего скора, но штраф и поощерение персонально мужского или женского скора в зависимости от перекоса который нужно вычислять тут же - вот это должно дать нам хороший результат я чувствую

In [ ]:
# этот код мы реализуем тут 

# Анализ ошибок тоже может помочь!) займемся им позже

In [43]:
# Создание DataFrame с ошибочными предсказаниями
df_errors = df_transposed_val[df_transposed_val['target'] != df_transposed_val['predict_gender']]

In [44]:
# Вывод первых нескольких строк DataFrame с ошибочными предсказаниями
df_errors.head()

,user_id,target,exchange-sessions,orders,visits,site-meta,last-visits-in-categories,orders_count,visits_count,last_visited_categories,predict_gender
8,user_127764,male,None,"[{'site-id': 179, 'orders': [{'created-at': 13...","[{'site-id': 3, 'first-seen': 1690804257, 'las...","[{'site-id': 179, 'recency': 1, 'frequency': 2...","[{'category': 'fashion', 'last-visit-at': 1685...","{179: 2, 40: 1}","{3: 2, 13: 1, 2: 1, 208: 2, 85: 2, 431: 6, 7: 1}","[fashion, other, bank, household_appliances, f...",female
9,user_127765,female,None,"[{'site-id': 16, 'orders': [{'created-at': 156...","[{'site-id': 3, 'first-seen': 1684519666, 'las...","[{'site-id': 2}, {'site-id': 16, 'recency': 1,...","[{'category': 'other', 'last-visit-at': 169651...","{16: 1, 2: 6, 691: 2}",{3: 4},[other],male
14,user_127770,female,None,"[{'site-id': 39, 'orders': [{'created-at': 167...","[{'site-id': 13, 'first-seen': 1684836326, 'la...","[{'site-id': 2}, {'site-id': 39, 'recency': 1,...","[{'category': 'bank', 'last-visit-at': 1674136...","{39: 1, 244: 1, 1: 1}","{13: 1, 3: 1}","[bank, electronics, other, media, cosmetics, f...",male
15,user_127771,male,"[{'landed-at': 1698678911, 'sites': [169, 2, 2...","[{'site-id': 62, 'orders': [{'created-at': 168...","[{'site-id': 34, 'first-seen': 1685510491, 'la...","[{'site-id': 42}, {'site-id': 33}, {'site-id':...","[{'category': 'fashion', 'last-visit-at': 1698...","{62: 1, 40: 1, 209: 2, 29: 5, 74: 1}","{34: 2, 135: 2, 149: 1, 26: 4, 110: 2, 713: 1,...","[fashion, other, bank, electronics, cosmetics,...",female
20,user_127776,male,"[{'landed-at': 1697833017, 'sites': [2, 169, 7...","[{'site-id': 49, 'orders': [{'created-at': 150...","[{'site-id': 912, 'first-seen': 1688017479, 'l...","[{'site-id': 495}, {'site-id': 49, 'recency': ...","[{'category': 'other', 'last-visit-at': 169783...","{49: 10, 191: 1, 34: 1, 112: 6, 21: 1, 39: 1, ...","{912: 2, 495: 5, 66: 1, 178: 1, 357: 1, 51: 1,...","[other, bank, travel, pets, food, electronics,...",female
